# LN - Data PP - Stability and efficiency calculations

## Import libraries and data

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import networkx as nx
import itertools
import matplotlib.pyplot as plt
import time
import pickle
import deepdish as dd
import h5py
import os

from tqdm.notebook import trange, tqdm
from time import sleep

from dask_cloudprovider import FargateCluster
from dask.distributed import Client
import dask.array as da
import dask

In [2]:
directory='/Users/dsrincon/Dropbox/Personal/EDU/Posgrado/masters_usa/MIMS/research/MIMS_capstone'

In [3]:
# Load Data

# Dataframe
decisions_df=pd.read_csv(directory+'/data/LN_channels.csv')

# Channel closures
with open(directory+'/data/channel_closures.p', 'rb') as pickle_file:
    channel_closures = pickle.load(pickle_file)
    
    
# Channel openings     
with open(directory+'/data/channel_opens.p', 'rb') as pickle_file:
    channel_opens = pickle.load(pickle_file)
    

extraction_id=1585344554
graph_dir=directory+'/data/graph_snapshots/'+str(extraction_id)+'_connected'



In [4]:
decisions_df.head()

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,close_time,close_fee,last_update,close_type,close_htlc_count,close_balance_a,close_balance_b,dec_id,node0_id,node1_id
0,505149x622x0,505149,f6bc767df9148ebf76d2b9baf4eb46e3230712c2bf5a51...,bc1qjmg6ev344fenh3zhg0yjl6hyvxpxluw6x9nn2a5lv4...,592638.0,82cb2ea2a06c8c453d8b9ca08e17bbefe87225aa380b2d...,0250373555232cec757ea141273e75381c84cc3ab22f1e...,02ef61a252f9504a42fc264a28476f44cea0711a44b2da...,300000,2019-08-22 02:49:00,...,1.567276e+09,184.0,1.563172e+09,mutual,0.0,3570.0,296246.0,0,3098,1492
1,513675x2245x0,513675,4297b5fe9beeb701c67fd0f84861b22edbcafe5c25be67...,bc1qymmdt0vzhdjqyqw2cevrqppp6rrlg5j2l20yk72z6y...,594718.0,3f86d9427c750f37a963b5a329da8941520f5a6cdbfe02...,028aa5a991a2acf33da91674fe062219b640e5e57d77a4...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,50000,2019-10-07 02:42:58,...,1.568401e+09,4410.0,1.552879e+09,unused,0.0,45590.0,0.0,1,5474,7365
2,513887x1177x0,513887,3b4cc434e62c1739e79171c7c1641bf9ac0e32d8530c68...,bc1q48l3h7sfdjaqat3sy98naltkwlujwefwnkfqxfm8fd...,NaN,NaN,02d97e94cfeedca2a3da47acb400bc6836e671b3cb3fc0...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,50000,2020-02-14 03:15:22,...,NaN,NaN,1.581536e+09,NaN,NaN,NaN,NaN,2,2797,7365
3,513909x1248x0,513909,86311514680351b1e644276efd7704ba13be169cc1a272...,bc1q24kvd9wdjdhwgr54fmu7cu9xldmsjuwdsq2ph5fwnj...,NaN,NaN,02ad6fb8d693dc1e4569bcedefadf5f72a931ae027dc0f...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,20000,2020-02-14 03:15:26,...,NaN,NaN,1.581464e+09,NaN,NaN,NaN,NaN,3,211,7365
4,513910x1814x1,513910,7f010765ce336d2be78c846844544e6a06ce2c59e7785f...,bc1qzh9xrpqvyse7fuanc8tl5e75qymq5fzk3deh78hs02...,NaN,NaN,02ad6fb8d693dc1e4569bcedefadf5f72a931ae027dc0f...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,20000,2020-02-14 03:15:26,...,NaN,NaN,1.581466e+09,NaN,NaN,NaN,NaN,4,211,7365


## Setup Fargate Cluster

In [5]:
cluster = FargateCluster(n_workers=1,scheduler_timeout='30 minutes',image='dsrincon/dask-graph:03-28-20')

/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/contextlib.py:88: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources on AWS. Hang tight! 
  next(self.gen)


In [6]:
cluster

In [7]:
client = Client(cluster)

In [11]:
# Lazy extract Graphs
graph_snapshots=[]
for filename in sorted(os.listdir(graph_dir))[700:705]:
    G = nx.read_gpickle(graph_dir+'/'+filename)
    G_bet=nx.algorithms.centrality.approximate_current_flow_betweenness_centrality(G)
    graph_snapshots.append(G_bet)

In [13]:
print(graph_snapshots[0])

{3098: 0.0, 1492: 0.0, 5407: 0.0039784768563331996, 5432: 0.0, 6779: 0.10633304639393353, 2471: 0.0, 3325: 0.0731160595736515, 5357: 0.004483099738829517, 1047: 0.004483099738829551, 1341: 8.572043445010537e-18, 5395: 5.1432260670063215e-17, 1538: 0.0, 6244: 0.0, 4983: 0.0, 6139: 0.0, 2594: 0.0, 5885: 0.0, 1969: 0.0, 2134: 0.0, 1465: 0.0, 144: 0.0, 6514: 0.0, 405: 0.0, 228: 0.007292394380342393, 105: 0.0, 5934: 0.0, 2205: 0.0, 49: 0.0, 6031: 0.0, 5043: 0.0, 1580: 0.007254609431073918, 6752: 0.0, 2787: 0.0, 1962: 0.0, 3728: 0.0, 3935: 0.0, 1147: 0.1338605967132879, 601: 0.0, 3016: 0.0, 3001: 0.0, 575: 0.0, 5185: 0.014153919257105565, 1560: 0.003968724710100217, 1812: 0.04696248599535319, 4992: 0.008782130232681207, 1837: 0.0055575095372802355, 1853: 0.0, 7516: 0.00887331045405059, 1170: 0.0, 7573: 0.0, 2718: 0.0, 2799: 0.0, 3277: 0.05167620134549839, 6022: 0.0, 5474: 0.0, 7365: 0.0, 1622: 0.0, 6292: 0.0, 2797: 0.0, 211: 0.24108396229013052, 2166: 0.017000754474047742, 6978: 0.0242979194

In [10]:
#lens=[len(g) for g in graph_snapshots if len(g)>2]
print(len(lens))
#fig,ax1=plt.subplots(figsize=(8,8))
#ax1.plot(blocks_analyzed, snapshot_nodes)

307


## Stability/Efficiency analysis by utility definition

In order to understand the potential motivations behind each decision we analyze each decission (opening or closure of a channel) independently from the perspective of each of the participants in the decission, which we'll call the node under analysis. For each decission we extract or compute the following information: 



## Betweeness


Betweenness centrality measures how central is a network to the flow of information in a network. In the case of the Lightning Network the higher the betweenness centrality of a node, the more transactions (messages) that are routed through it. In particular, we will use a measure of betweenness centrality defined in (Brandes and Fleischer 2005 - https://link.springer.com/chapter/10.1007/978-3-540-31856-9_44) that models infomation through a network, as electric current, efficiently and not only considering shortest path. This allows us to account for the fact that not all transactions travel through shortes path given that there are fee and capacity considerations.  

### Baseline betweeness

In [9]:
#def current_betweenness(G):
 #   bet=nx.algorithms.centrality.approximate_current_flow_betweenness_centrality(G)
  #  return bet

snapshot_bet=[]

for g in graph_snapshots:
    g_bet=dask.delayed(nx.algorithms.centrality.approximate_current_flow_betweenness_centrality)(g)
    snapshot_bet.append(g_bet)

futures = dask.persist(*snapshot_bet)
    

distributed.protocol.pickle - INFO - Failed to deserialize b'\x80\x04\x95d\x03\x00\x00\x00\x00\x00\x00\x8c\x16tblib.pickling_support\x94\x8c\x12unpickle_exception\x94\x93\x94(\x8c\x08builtins\x94\x8c\x11FileNotFoundError\x94\x93\x94K\x02\x8c\x19No such file or directory\x94\x8c\x8e/Users/dsrincon/Dropbox/Personal/EDU/Posgrado/masters_usa/MIMS/research/MIMS_capstone/data/graph_snapshots/1585344554_connected/521473.gpickle\x94\x87\x94Nh\x00\x8c\x12unpickle_traceback\x94\x93\x94\x8c\x05tblib\x94\x8c\x05Frame\x94\x93\x94)\x81\x94}\x94(\x8c\tf_globals\x94}\x94(\x8c\x08__name__\x94\x8c\x12distributed.worker\x94\x8c\x08__file__\x94\x8c</opt/conda/lib/python3.7/site-packages/distributed/worker.py\x94u\x8c\x06f_code\x94h\x0b\x8c\x04Code\x94\x93\x94)\x81\x94}\x94(\x8c\x0bco_filename\x94\x8c</opt/conda/lib/python3.7/site-packages/distributed/worker.py\x94\x8c\x07co_name\x94\x8c\x0eapply_function\x94ububM2\rh\x0b\x8c\tTraceback\x94\x93\x94)\x81\x94}\x94(\x8c\x08tb_frame\x94h\r)\x81\x94}\x94(h\x10}

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

In [10]:
client.get_versions(check=True)

TypeError: sum() can't sum strings [use ''.join(seq) instead]

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

...

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=4 threads=16, memory=64.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=3 threads=12, memory=48.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=3 threads=12, memory=48.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=3 threads=12, memory=48.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/client.py", line 1100, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://172.31.59.63:8786' processes=3 threads=12, memory=48.00 GB>>
Traceback (most recent call last):
  File "/Users/dsrincon/anaconda/envs/graphanalysis/lib/python3.6/site-packages/torn

### Pairwise stability 

- **Marginal % change in betweenness (bet_mar_deltai)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting the decission (adding or removing a channel). Weighted current betweenness centrality is used for this measure.

In [ ]:
with tqdm(total=len(os.listdir(graph_dir))) as pbar:
    
    for filename in sorted(os.listdir(graph_dir)):
        
        # Read graph and extract block number
        G = nx.read_gpickle(graph_dir+'/'+filename)
        block=G.graph['block']
        
        # Check for closure decisions
        for dec in channel_closures[block]:
            
        



- **Actual % change in betweenness (bet_act_deltai)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted current betweenness centrality is used for this measure.

- **Marginal betweeness pairwise stability (bet_mar_pairstab)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a betweenness perspective.

- **Actual betweeness pairwise stability (bet_act_pairstab)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a betweenness perspective. 




### Nash stability 

- **% Change with respect to not making decision (bet_binstat_deltai)**: The % change in betwewnness centrality, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (bet_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher betweenness centrality. This tells me if my strategy helped me be better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (bet_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher betwneenness centrality. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (bet_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower betwneenness centrality. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average betweenness centrality for all the nodes. 



## Connectivity

### Pairwise stability 

- **Marginal % change in connectivity (con_mar_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting the decission (adding or removing a channel). Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Actual % change in connectivity (con_act_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Marginal connectivity pairwise stability (con_mar_pairstab)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a connectivity perspective.

- **Actual connectivity pairwise stability (con_act_pairstab)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a connectivity perspective.  



### Nash stability 

- **% Change with respect to not making decision (con_binstat_deltai)**: The % change in shortest path average, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (con_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher shortest path average. NOTE: This indicates if the strategy selected made the node better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (con_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher shortest path average. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (con_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower shortest path average. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average shortest path average for all the nodes. 



